In [106]:
import pandas as pd
from sklearn import tree

train = pd.read_csv('titanic/train.csv')
test = pd.read_csv('titanic/test.csv')
train_test_data = [train, test]

for dataset in train_test_data:
    # 결측치 처리
    dataset['Age'].fillna(dataset['Age'].median(), inplace=True)
    dataset['Embarked'].fillna('S', inplace=True)
    if dataset is test:
        dataset.loc[dataset['Fare'].isnull(), 'Fare'] = dataset[(dataset.Pclass == 3) & (dataset.Embarked == 'S')].Fare.median()
    
    # 새로운 열 추가
    dataset['Family Size'] = dataset.SibSp + dataset.Parch
    
    # 필요없는 열 삭제
    dataset.drop(['Name', 'Ticket', 'Cabin', 'SibSp', 'Parch'], axis=1, inplace=True)

    # 문자열 데이터를 수치로 변환
    dataset['Sex'] = dataset['Sex'].map({'male': 0, 'female': 1})
    dataset['Embarked'] = dataset['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})

train.loc[:, 'Pclass':]
test.loc[:, 'Pclass':]

target = train.Survived
model = tree.DecisionTreeClassifier(random_state=20240614).fit(train.loc[:, 'Pclass':], target)
pred = model.predict(test.loc[:, 'Pclass':])

pred

submission = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Survived': pred
})

submission

submission.to_csv('titanic_submission.csv', index=False)